# Strands Agents SDKとClaude 4 Interleaved Thinkingを使用したAgents as Tools

このノートブックでは、Strands Agents SDKとClaude 4の**インターリーブド思考（interleaved thinking）**機能を使用して、専門エージェントによるインテリジェントワークフローを調整する方法を示します。

## インターリーブド思考の理解

### インターリーブド思考とは？

インターリーブド思考は、Claude 4モデルの新機能で、以下のことが可能になります：

1. **ツール呼び出し間で考える**: 次のステップを決定する前に結果を処理し推論します
2. **推論を伴う複数ツールのチェーン化**: 洗練されたマルチステップの意思決定を行います
3. **戦略を動的に適応**: 中間結果に基づいてアプローチを変更します

### 仕組み

インターリーブド思考を使用する場合としない場合のエージェントのイベントループには多くの類似点があります：

クエリ → LLMが思考 → LLMがツール呼び出しを決定 → イベントループがツールを呼び出す → 出力がLLMに返される → [LLMがツールを呼び出す必要がなくなるまで継続 - 最終回答をレンダリング]

インターリーブド思考で気づく主な違いは、イベントループがLLMの「決定」ではなく「思考」に基づいて動作することです。上記のループの2番目のリンク「思考」に注目してください。従来のイベントループでは、思考は隠されています。LLMがツール呼び出しの決定をレンダリングするか、最終回答を生成するまで待つ必要があります。

インターリーブド思考の場合、LLMは第2ステップ「LLMが思考中」にいる間に、その思考をイベントループに「漏らし」ており、イベントループはLLMがそれについて「考える」とすぐにツールを実行するように設定されています。これは、LLMが考え終わる頃には、実際に最初の「決定」で最終回答を持っていることを意味します。

### インターリーブド思考の有効化

StrandsとBedrockでこの機能を有効にするには：

- `temperature=1`を設定（思考が有効な場合は必須）
- ベータヘッダーを追加：`"anthropic_beta": ["interleaved-thinking-2025-05-14"]`
- 推論バジェットを設定：`"reasoning_config": {"type": "enabled", "budget_tokens": 3000}`

## セットアップとインポート

In [ ]:
import os
from strands import Agent, tool
from strands.models import BedrockModel
from strands.models import bedrock

bedrock.DEFAULT_BEDROCK_MODEL_ID = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"

## 専門エージェントをツールとして定義

まず、Strandsの`@tool`デコレーターを使用して4つの専門エージェントを作成します：

- **Researcher（研究者）**: 事実情報を収集
- **Data Analyst（データアナリスト）**: 情報を処理および分析
- **Fact Checker（ファクトチェッカー）**: 情報の正確性を検証
- **Report Writer（レポートライター）**: 洗練された最終ドキュメントを作成

In [ ]:
# Strands @toolデコレーターを使用してツールとして実装された専門エージェント
@tool
def researcher(query: str) -> str:
    """
    事実情報を収集する研究専門家。
    
    Args:
        query: 調査する研究質問またはトピック
        
    Returns:
        研究結果とソース
    """
    # 焦点を絞った研究エージェントを作成
    # 注意：各呼び出しは新しいエージェントインスタンスを作成します（ステートレス）
    research_agent = Agent(
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        system_prompt="あなたは研究専門家です。事実情報を収集し、可能な限りソースを引用してください。応答は200語以内に抑えてください。",
        callback_handler=None  # ツールエージェントにはストリーミングなし
    )
    
    # 研究タスクを実行
    result = research_agent(f"Research: {query}")
    return str(result)

In [ ]:
@tool
def data_analyst(data: str) -> str:
    """
    情報を処理および分析するデータアナリスト。
    
    Args:
        data: 分析する生データまたは研究結果
        
    Returns:
        洞察とパターンを含む分析
    """
    # アナリストエージェントは洞察の抽出に焦点を当てる
    analysis_agent = Agent(
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        system_prompt="あなたはデータアナリストです。主要な洞察を抽出し、パターンを特定し、分析的結論を提供してください。実行可能な洞察に焦点を当ててください。",
        callback_handler=None
    )
    
    # 提供されたデータを分析
    result = analysis_agent(f"このデータを分析して洞察を提供してください: {data}")
    return str(result)

In [ ]:
@tool
def fact_checker(information: str) -> str:
    """
    情報の正確性を検証するファクトチェッカー。
    
    Args:
        information: 検証する主張またはデータ
        
    Returns:
        正確性評価を含むファクトチェック結果
    """
    # 検証用のファクトチェックエージェント
    fact_check_agent = Agent(
        model="us.anthropic.claude-3-7-sonnet-20250219-v1:0",
        system_prompt="あなたはファクトチェッカーです。主張を検証し、信頼性を評価し、信頼度レベルを提供してください。疑わしい記述を特定してください。",
        callback_handler=None
    )
    
    # 情報を検証
    result = fact_check_agent(f"この情報をファクトチェックしてください: {information}")
    return str(result)

In [ ]:
@tool
def report_writer(analysis: str) -> str:
    """
    洗練された最終ドキュメントを作成するレポートライター。
    
    Args:
        analysis: 分析されたデータと洞察
        
    Returns:
        フォーマットされた最終レポート
    """
    # プロフェッショナルな出力のためのライターエージェント
    writer_agent = Agent(
        system_prompt="あなたはプロフェッショナルなレポートライターです。エグゼクティブサマリーと実行可能な推奨事項を含む、明確で適切に構造化されたレポートを作成してください。",
        callback_handler=None
    )
    
    # レポートを作成
    result = writer_agent(f"次の内容に基づいてプロフェッショナルなレポートを作成してください: {analysis}")
    return str(result)

## インターリーブド思考を備えたClaude 4オーケストレーター

次に、オーケストレーターを作成します - インターリーブド思考を使用して専門エージェントをインテリジェントに調整するClaude 4エージェントです。

### オーケストレーターの仕組み：

1. ユーザーから高レベルのタスクを受け取ります
2. 必要な情報について**考えます**
3. 研究者ツールを呼び出して初期データを収集します
4. 研究結果と必要な分析について**考えます**
5. データアナリストを呼び出して調査結果を処理します
6. 正確性と検証のニーズについて**考えます**
7. 必要に応じてファクトチェッカーを呼び出します
8. 調査結果の提示方法について**考えます**
9. 最終出力のためにレポートライターを呼び出します
10. 応答前に完全なワークフローについて**振り返ります**

In [ ]:
# Strandsを使用したインターリーブド思考を備えたClaude 4オーケストレーター
class StrandsInterlevedWorkflowOrchestrator:
    def __init__(self):
        # インテリジェントワークフロー調整のためのオーケストレーターシステムプロンプトを定義
        self.system_prompt = """あなたは専門エージェントにアクセスできるインテリジェントワークフローオーケストレーターです。

        あなたの役割は、これらの専門エージェントを使用してワークフローをインテリジェントに調整することです：
        - researcher: あらゆるトピックに関する事実情報を収集
        - data_analyst: データを分析し洞察を抽出
        - fact_checker: 情報の正確性を検証
        - report_writer: 洗練された最終レポートを作成

        """
    
    def run_workflow(self, task: str, enable_interleaved_thinking: bool = True) -> str:
        """指定されたタスクのインテリジェントワークフローを実行します。
        
        Args:
            task: 完了するタスク
            enable_interleaved_thinking: インターリーブド思考を有効にするかどうか（デフォルト: True）
        
        オーケストレーターは以下を行います：
        1. タスク要件を理解
        2. 最適なアプローチについて考える
        3. 専門エージェントを調整
        4. ステップ間で結果を振り返る
        5. 包括的な出力を生成
        """
        thinking_mode = "インターリーブド思考あり" if enable_interleaved_thinking else "インターリーブド思考なし"
        print(f"\n{thinking_mode}でインテリジェントワークフローを開始: {task}")
        print("=" * 70)
        
        # Bedrock経由でインターリーブド思考の有無でClaude 4を設定
        if enable_interleaved_thinking:
            claude4_model = BedrockModel(
                model_id="us.anthropic.claude-sonnet-4-20250514-v1:0",
                max_tokens=4096,
                temperature=1,  # 思考が有効な場合は1である必要があります
                additional_request_fields={
                    # インターリーブド思考ベータ機能を有効化
                    "anthropic_beta": ["interleaved-thinking-2025-05-14"],
                    # 推論パラメーターを設定
                    "reasoning_config": {
                        "type": "enabled",  # 思考をオン
                        "budget_tokens": 3000  # 思考トークンバジェット
                    }
                }
            )
        else:
            claude4_model = BedrockModel(
                model_id="us.anthropic.claude-sonnet-4-20250514-v1:0",
                max_tokens=4096,
                temperature=1
            )
        
        # Claude 4と専門ツールを使用してオーケストレーターエージェントを作成
        orchestrator = Agent(
            model=claude4_model,
            system_prompt=self.system_prompt,
            tools=[researcher, data_analyst, fact_checker, report_writer]
        )
        
        prompt = f"""インテリジェントワークフロー調整を使用してこのタスクを完了してください: {task}

        指示：
        1. このタスクを達成するために必要な情報について慎重に考えてください
        2. 専門エージェントを戦略的に使用してください - それぞれ独自の強みがあります
        3. 各ツール使用後、結果を振り返りアプローチを適応させてください
        4. 包括的な結果のために必要に応じて複数のエージェントを調整してください
        5. 適切な場合はファクトチェックにより正確性を確保してください
        6. すべての側面に対応する包括的な最終応答を提供してください
        
        覚えておいてください：ツール呼び出し間の思考は、より良い意思決定に役立ちます。
        計画、結果の評価、戦略の調整にそれを使用してください。
        """
        
        try:
            result = orchestrator(prompt)
            return str(result)
        except Exception as e:
            return f"ワークフロー失敗: {e}"

## デモの実行

オーケストレーターの動作を見てみましょう！思考中にどのように考え、ツールを呼び出しているかを観察してください。

In [ ]:
# オーケストレーターを作成
print("Strands Agents SDK: Claude 4インターリーブド思考ワークフローデモ")
print("=" * 70)

try:
    orchestrator = StrandsInterlevedWorkflowOrchestrator()
    print("✅ オーケストレーターの初期化に成功しました！")
except Exception as e:
    print(f"❌ オーケストレーターの初期化に失敗しました: {e}")

In [ ]:
# テストケースでワークフローを実行
test_case = "リモートワークが生産性に与える影響を分析し、戦略的推奨事項を提供してください"

print(f"📋 タスク: {test_case}\n")

try:
    result = orchestrator.run_workflow(test_case)
    
    print(f"\n📊 ワークフロー結果:")
    print("=" * 70)
    print(result)
except Exception as e:
    print(f"❌ ワークフロー実行に失敗しました: {e}")

## インターリーブド思考なしで試す

オーケストレーターを呼び出し、インターリーブド思考を無効にして実験してください。出力の違いを観察してください。

In [ ]:
# 同じタスクをインターリーブド思考なしで試してみましょう
print("\n" + "="*70)
print("🔄 インターリーブド思考なしで同じタスクを実行中")
print("="*70)

try:
    result_without_thinking = orchestrator.run_workflow(test_case, enable_interleaved_thinking=False)
    
    print(f"\n📊 ワークフロー結果（インターリーブド思考なし）:")
    print("=" * 70)
    print(result_without_thinking)
except Exception as e:
    print(f"❌ ワークフロー実行に失敗しました: {e}")